In [5]:
from pyomo.environ import *

import qiskit
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo

In [13]:
project_selection_model = ConcreteModel()

#Problem data definition:
projects = [1, 2, 3, 4, 5]
returns = {1: 20, 2: 40, 3: 20, 4: 15, 5: 30}
cost = {
    1: [5, 5, 8],
    2: [4, 7, 10],
    3: [3, 9, 2],
    4: [7, 4, 3],
    5: [8, 6, 10]
}

annual_budget = [25, 25, 25]

#Decision variables
project_selection_model.x = Var(projects, within=Binary)

#OBJECTIVE FUNCTION: maximize total return
project_selection_model.obj = Objective(
    expr=sum(returns[j] * project_selection_model.x[j] for j in projects),
    sense=maximize
)


In [14]:
#Applying the restrictions
project_selection_model.ano1_constraint = Constraint(
    expr=sum(cost[j][0] * project_selection_model.x[j] for j in projects) <= annual_budget[0]
)
project_selection_model.ano2_constraint = Constraint(
    expr=sum(cost[j][1] * project_selection_model.x[j] for j in projects) <= annual_budget[1]
)
project_selection_model.y3_constraint = Constraint(
    expr=sum(cost[j][2] * project_selection_model.x[j] for j in projects) <= annual_budget[2]
)

In [16]:
solver = SolverFactory('glpk')
result = solver.solve(project_selection_model)

print("Optimization Status:", result.solver.status)
print("Solution Status:", result.solver.termination_condition)

print("\nSelected Projects:")
for j in projects:
    if project_selection_model.x[j].value == 1:
        print(f"Project {j}: Selected")

print("\nTotal Return:", project_selection_model.obj())

Optimization Status: ok
Solution Status: optimal

Selected Projects:
Project 1: Selected
Project 2: Selected
Project 3: Selected
Project 4: Selected

Total Return: 95.0
